# FPSci Latency Study - Statistical Analysis

This notebook performs statistical analyses on the processed experimental data.

## Analyses

1. Descriptive statistics
2. Correlation analysis (latency vs performance, latency vs QoE)
3. Mixed-effects models (within-subjects repeated measures)
4. Effect sizes and pairwise comparisons


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols, mixedlm
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Processed Data


In [4]:
from pathlib import Path

DATA_DIR = Path('../analysis/processed_data')
OUTPUT_DIR = Path('../analysis/results')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load combined data
df = pd.read_csv(DATA_DIR / 'combined_data.csv')

print(f"Loaded {len(df)} observations")
print(f"Participants: {len(df['participant_id'].unique())}")
print(f"Games: {df['game_display'].unique()}")
print(f"Latency conditions: {sorted(df['latency_ms'].unique())}")

df.head()

Loaded 300 observations
Participants: 17
Games: <StringArray>
['Dave the Diver', 'Half-Life 2', 'Fitts Law', 'Feeding Frenzy',
 'Rocket League']
Length: 5, dtype: str
Latency conditions: [np.float64(0.0), np.float64(75.0), np.float64(150.0), np.float64(225.0)]


,participant_id,game,latency_ms,score,mean_time_ms,median_time_ms,error_rate,throughput,n_trials,goals,shots,saves,assists,quality_rating,acceptable,game_display,score_z,score_pct_of_baseline
0,1,dave_the_diver,150.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,Dave the Diver,-0.058850,100.000000
1,1,dave_the_diver,0.0,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,Dave the Diver,-0.107517,90.000000
2,1,dave_the_diver,75.0,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,Dave the Diver,-0.139961,83.333333
3,1,dave_the_diver,225.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,2.0,Dave the Diver,-0.383293,33.333333
4,1,half_life_2,75.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,1.0,Half-Life 2,0.789281,100.000000


## 2. Descriptive Statistics


In [5]:
print("=" * 80)
print("DESCRIPTIVE STATISTICS BY LATENCY CONDITION")
print("=" * 80)

# Overall statistics by latency
descriptive_latency = df.groupby('latency_ms').agg({
    'score': ['count', 'mean', 'std', 'min', 'max'],
    'score_z': ['mean', 'std'],
    'quality_rating': ['mean', 'std'],
    'acceptable': ['mean', 'sum']
}).round(3)

print(descriptive_latency)

# Save to file
descriptive_latency.to_csv(OUTPUT_DIR / 'descriptive_stats_by_latency.csv')

print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS BY GAME")
print("=" * 80)

descriptive_game = df.groupby('game_display').agg({
    'score': ['count', 'mean', 'std', 'min', 'max'],
    'quality_rating': ['mean', 'std'],
    'acceptable': ['mean']
}).round(3)

print(descriptive_game)
descriptive_game.to_csv(OUTPUT_DIR / 'descriptive_stats_by_game.csv')

print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS BY GAME AND LATENCY")
print("=" * 80)

descriptive_game_latency = df.groupby(['game_display', 'latency_ms']).agg({
    'score': ['count', 'mean', 'std'],
    'quality_rating': ['mean', 'std'],
    'acceptable': 'mean'
}).round(3)

print(descriptive_game_latency)
descriptive_game_latency.to_csv(OUTPUT_DIR / 'descriptive_stats_by_game_latency.csv')

DESCRIPTIVE STATISTICS BY LATENCY CONDITION
           score                               score_z        quality_rating  \
           count     mean      std  min    max    mean    std           mean   
latency_ms                                                                     
0.0           76   94.610  232.591  0.0  970.0   0.514  1.059          4.533   
75.0          75  129.327  273.575  0.0  970.0   0.313  0.819          3.749   
150.0         75   92.669  187.747  0.0  980.0  -0.176  0.987          2.803   
225.0         74   49.027   86.333  0.0  320.0  -0.667  0.602          2.043   

                  acceptable         
              std       mean    sum  
latency_ms                           
0.0         0.738      1.039   79.0  
75.0        0.995      1.200   90.0  
150.0       1.309      1.446  107.0  
225.0       1.075      1.662  123.0  

DESCRIPTIVE STATISTICS BY GAME
               score                                  quality_rating         \
               cou

## 3. Correlation Analysis

Examine correlations between latency and outcomes (performance, QoE).


In [ ]:
print("=" * 80)
print("CORRELATION ANALYSIS: LATENCY vs OUTCOMES")
print("=" * 80)

correlation_results = []

# Overall correlations
print("\nOVERALL (All Games Combined):")
print("-" * 80)

# Latency vs Z-scored performance
r_perf, p_perf = pearsonr(df['latency_ms'], df['score_z'])
rho_perf, p_rho_perf = spearmanr(df['latency_ms'], df['score_z'])
print(f"Latency vs Performance (z-score):")
print(f"  Pearson r = {r_perf:.3f}, p = {p_perf:.4f}")
print(f"  Spearman ρ = {rho_perf:.3f}, p = {p_rho_perf:.4f}")

correlation_results.append({
    'analysis': 'Overall',
    'game': 'All',
    'outcome': 'Performance (z)',
    'pearson_r': r_perf,
    'pearson_p': p_perf,
    'spearman_rho': rho_perf,
    'spearman_p': p_rho_perf,
    'n': len(df)
})

# Latency vs QoE (quality rating)
qoe_valid = df.dropna(subset=['quality_rating'])
r_qoe, p_qoe = pearsonr(qoe_valid['latency_ms'], qoe_valid['quality_rating'])
rho_qoe, p_rho_qoe = spearmanr(qoe_valid['latency_ms'], qoe_valid['quality_rating'])
print(f"\nLatency vs QoE Quality Rating:")
print(f"  Pearson r = {r_qoe:.3f}, p = {p_qoe:.4f}")
print(f"  Spearman ρ = {rho_qoe:.3f}, p = {p_rho_qoe:.4f}")

correlation_results.append({
    'analysis': 'Overall',
    'game': 'All',
    'outcome': 'QoE Quality',
    'pearson_r': r_qoe,
    'pearson_p': p_qoe,
    'spearman_rho': rho_qoe,
    'spearman_p': p_rho_qoe,
    'n': len(qoe_valid)
})

# Latency vs Acceptability
accept_valid = df.dropna(subset=['acceptable'])
rho_accept, p_rho_accept = spearmanr(accept_valid['latency_ms'], accept_valid['acceptable'])
print(f"\nLatency vs Acceptability (binary):")
print(f"  Spearman ρ = {rho_accept:.3f}, p = {p_rho_accept:.4f}")

correlation_results.append({
    'analysis': 'Overall',
    'game': 'All',
    'outcome': 'Acceptability',
    'pearson_r': np.nan,
    'pearson_p': np.nan,
    'spearman_rho': rho_accept,
    'spearman_p': p_rho_accept,
    'n': len(accept_valid)
})

# Performance vs QoE
perf_qoe_valid = df.dropna(subset=['score_z', 'quality_rating'])
r_perf_qoe, p_perf_qoe = pearsonr(perf_qoe_valid['score_z'], perf_qoe_valid['quality_rating'])
print(f"\nPerformance vs QoE Quality:")
print(f"  Pearson r = {r_perf_qoe:.3f}, p = {p_perf_qoe:.4f}")

# Per-game correlations
print("\n" + "=" * 80)
print("PER-GAME CORRELATIONS")
print("=" * 80)

for game in sorted(df['game_display'].unique()):
    game_df = df[df['game_display'] == game]

    print(f"\n{game}:")
    print("-" * 80)

    # Latency vs Score (raw)
    if len(game_df) > 2:
        r_g, p_g = pearsonr(game_df['latency_ms'], game_df['score'])
        rho_g, p_rho_g = spearmanr(game_df['latency_ms'], game_df['score'])
        print(f"  Latency vs Score: r={r_g:.3f} (p={p_g:.4f}), ρ={rho_g:.3f} (p={p_rho_g:.4f})")

        correlation_results.append({
            'analysis': 'Per-game',
            'game': game,
            'outcome': 'Performance',
            'pearson_r': r_g,
            'pearson_p': p_g,
            'spearman_rho': rho_g,
            'spearman_p': p_rho_g,
            'n': len(game_df)
        })

    # Latency vs QoE
    game_qoe = game_df.dropna(subset=['quality_rating'])
    if len(game_qoe) > 2:
        r_qoe_g, p_qoe_g = pearsonr(game_qoe['latency_ms'], game_qoe['quality_rating'])
        print(f"  Latency vs QoE: r={r_qoe_g:.3f} (p={p_qoe_g:.4f})")

        correlation_results.append({
            'analysis': 'Per-game',
            'game': game,
            'outcome': 'QoE Quality',
            'pearson_r': r_qoe_g,
            'pearson_p': p_qoe_g,
            'spearman_rho': np.nan,
            'spearman_p': np.nan,
            'n': len(game_qoe)
        })

# Save correlation results
corr_df = pd.DataFrame(correlation_results)
corr_df.to_csv(OUTPUT_DIR / 'correlation_results.csv', index=False)
print(f"\nCorrelation results saved to: {OUTPUT_DIR / 'correlation_results.csv'}")

CORRELATION ANALYSIS: LATENCY vs OUTCOMES

OVERALL (All Games Combined):
--------------------------------------------------------------------------------
Latency vs Performance (z-score):
  Pearson r = -0.454, p = 0.0000
  Spearman ρ = -0.518, p = 0.0000

Latency vs QoE Quality Rating:
  Pearson r = -0.671, p = 0.0000
  Spearman ρ = -0.673, p = 0.0000

Latency vs Acceptability (binary):
  Spearman ρ = 0.501, p = 0.0000

Performance vs QoE Quality:
  Pearson r = 0.382, p = 0.0000

PER-GAME CORRELATIONS

Dave the Diver:
--------------------------------------------------------------------------------
  Latency vs Score: r=-0.097 (p=0.4858), ρ=-0.354 (p=0.0086)
  Latency vs QoE: r=-0.594 (p=0.0000)

Feeding Frenzy:
--------------------------------------------------------------------------------
  Latency vs Score: r=-0.301 (p=0.0125), ρ=-0.298 (p=0.0135)
  Latency vs QoE: r=-0.634 (p=0.0000)

Fitts Law:
--------------------------------------------------------------------------------
  Late

## 4. Repeated Measures ANOVA

Test if latency significantly affects performance and QoE.


In [ ]:
print("=" * 80)
print("REPEATED MEASURES ANOVA")
print("=" * 80)

anova_results = []

# ANOVA for Performance (z-score) - Overall
print("\nPerformance (z-score) by Latency - Overall:")
print("-" * 80)

# Simple one-way ANOVA (not accounting for repeated measures yet)
latency_groups = [df[df['latency_ms'] == lat]['score_z'].dropna() for lat in sorted(df['latency_ms'].unique())]
f_stat, p_val = f_oneway(*latency_groups)
print(f"F-statistic: {f_stat:.3f}")
print(f"p-value: {p_val:.4f}")

anova_results.append({
    'outcome': 'Performance (z)',
    'game': 'All',
    'f_statistic': f_stat,
    'p_value': p_val,
    'n_groups': len(latency_groups),
    'total_n': sum(len(g) for g in latency_groups)
})

# ANOVA for QoE Quality Rating
print("\nQoE Quality Rating by Latency - Overall:")
print("-" * 80)

qoe_valid = df.dropna(subset=['quality_rating'])
qoe_groups = [qoe_valid[qoe_valid['latency_ms'] == lat]['quality_rating'] for lat in sorted(qoe_valid['latency_ms'].unique())]
f_stat_qoe, p_val_qoe = f_oneway(*qoe_groups)
print(f"F-statistic: {f_stat_qoe:.3f}")
print(f"p-value: {p_val_qoe:.4f}")

anova_results.append({
    'outcome': 'QoE Quality',
    'game': 'All',
    'f_statistic': f_stat_qoe,
    'p_value': p_val_qoe,
    'n_groups': len(qoe_groups),
    'total_n': sum(len(g) for g in qoe_groups)
})

# Per-game ANOVAs
print("\n" + "=" * 80)
print("PER-GAME ANOVA RESULTS")
print("=" * 80)

for game in sorted(df['game_display'].unique()):
    game_df = df[df['game_display'] == game]

    print(f"\n{game}:")
    print("-" * 80)

    # Performance ANOVA
    game_groups = [game_df[game_df['latency_ms'] == lat]['score'].dropna() for lat in sorted(game_df['latency_ms'].unique())]
    if all(len(g) > 0 for g in game_groups):
        f_g, p_g = f_oneway(*game_groups)
        print(f"  Performance: F={f_g:.3f}, p={p_g:.4f}")

        anova_results.append({
            'outcome': 'Performance',
            'game': game,
            'f_statistic': f_g,
            'p_value': p_g,
            'n_groups': len(game_groups),
            'total_n': sum(len(g) for g in game_groups)
        })

    # QoE ANOVA
    game_qoe = game_df.dropna(subset=['quality_rating'])
    qoe_game_groups = [game_qoe[game_qoe['latency_ms'] == lat]['quality_rating'] for lat in sorted(game_qoe['latency_ms'].unique())]
    if all(len(g) > 0 for g in qoe_game_groups):
        f_qoe_g, p_qoe_g = f_oneway(*qoe_game_groups)
        print(f"  QoE Quality: F={f_qoe_g:.3f}, p={p_qoe_g:.4f}")

        anova_results.append({
            'outcome': 'QoE Quality',
            'game': game,
            'f_statistic': f_qoe_g,
            'p_value': p_qoe_g,
            'n_groups': len(qoe_game_groups),
            'total_n': sum(len(g) for g in qoe_game_groups)
        })

# Save ANOVA results
anova_df = pd.DataFrame(anova_results)
anova_df.to_csv(OUTPUT_DIR / 'anova_results.csv', index=False)
print(f"\nANOVA results saved to: {OUTPUT_DIR / 'anova_results.csv'}")

REPEATED MEASURES ANOVA

Performance (z-score) by Latency - Overall:
--------------------------------------------------------------------------------
F-statistic: 26.662
p-value: 0.0000

QoE Quality Rating by Latency - Overall:
--------------------------------------------------------------------------------
F-statistic: 80.803
p-value: 0.0000

PER-GAME ANOVA RESULTS

Dave the Diver:
--------------------------------------------------------------------------------
  Performance: F=0.755, p=0.5249
  QoE Quality: F=10.324, p=0.0000

Feeding Frenzy:
--------------------------------------------------------------------------------
  Performance: F=5.170, p=0.0029
  QoE Quality: F=14.952, p=0.0000

Fitts Law:
--------------------------------------------------------------------------------
  Performance: F=129.173, p=0.0000
  QoE Quality: F=99.690, p=0.0000

Half-Life 2:
--------------------------------------------------------------------------------
  Performance: F=8.416, p=0.0001
  QoE Quali

## 5. Post-hoc Pairwise Comparisons

Tukey HSD test for pairwise latency comparisons.


In [8]:
print("=" * 80)
print("POST-HOC PAIRWISE COMPARISONS (Tukey HSD)")
print("=" * 80)

# Performance (z-score)
print("\nPerformance (z-score) - All Games:")
print("-" * 80)
tukey_perf = pairwise_tukeyhsd(df['score_z'], df['latency_ms'])
print(tukey_perf)

# Save to file
with open(OUTPUT_DIR / 'tukey_performance.txt', 'w') as f:
    f.write(str(tukey_perf))

# QoE Quality
qoe_valid = df.dropna(subset=['quality_rating'])
print("\nQoE Quality Rating - All Games:")
print("-" * 80)
tukey_qoe = pairwise_tukeyhsd(qoe_valid['quality_rating'], qoe_valid['latency_ms'])
print(tukey_qoe)

with open(OUTPUT_DIR / 'tukey_qoe.txt', 'w') as f:
    f.write(str(tukey_qoe))

POST-HOC PAIRWISE COMPARISONS (Tukey HSD)

Performance (z-score) - All Games:
--------------------------------------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   0.0   75.0  -0.2009 0.5044 -0.5734  0.1716  False
   0.0  150.0  -0.6904    0.0 -1.0629 -0.3179   True
   0.0  225.0  -1.1817    0.0 -1.5555  -0.808   True
  75.0  150.0  -0.4895 0.0045 -0.8632 -0.1157   True
  75.0  225.0  -0.9808    0.0 -1.3558 -0.6058   True
 150.0  225.0  -0.4914 0.0045 -0.8663 -0.1164   True
----------------------------------------------------

QoE Quality Rating - All Games:
--------------------------------------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   0.0   75.0  -0.7836    0.0 -1.2239 

## 6. Effect Sizes

Calculate Cohen's d for pairwise latency comparisons.


In [ ]:
def cohens_d(group1, group2):
    """Calculate Cohen's d effect size."""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std

print("=" * 80)
print("EFFECT SIZES (Cohen's d)")
print("=" * 80)

effect_sizes = []
latencies = sorted(df['latency_ms'].unique())

# Performance effect sizes
print("\nPerformance (z-score) - Baseline (0ms) vs Higher Latencies:")
print("-" * 80)

baseline_perf = df[df['latency_ms'] == 0]['score_z'].dropna()

for lat in latencies[1:]:
    lat_perf = df[df['latency_ms'] == lat]['score_z'].dropna()
    d = cohens_d(baseline_perf, lat_perf)
    print(f"0ms vs {lat}ms: d = {d:.3f}")

    effect_sizes.append({
        'outcome': 'Performance (z)',
        'comparison': f'0ms vs {lat}ms',
        'cohens_d': d,
        'n_baseline': len(baseline_perf),
        'n_comparison': len(lat_perf)
    })

# QoE effect sizes
print("\nQoE Quality - Baseline (0ms) vs Higher Latencies:")
print("-" * 80)

baseline_qoe = df[df['latency_ms'] == 0]['quality_rating'].dropna()

for lat in latencies[1:]:
    lat_qoe = df[df['latency_ms'] == lat]['quality_rating'].dropna()
    d_qoe = cohens_d(baseline_qoe, lat_qoe)
    print(f"0ms vs {lat}ms: d = {d_qoe:.3f}")

    effect_sizes.append({
        'outcome': 'QoE Quality',
        'comparison': f'0ms vs {lat}ms',
        'cohens_d': d_qoe,
        'n_baseline': len(baseline_qoe),
        'n_comparison': len(lat_qoe)
    })

# Save effect sizes
effect_df = pd.DataFrame(effect_sizes)
effect_df.to_csv(OUTPUT_DIR / 'effect_sizes.csv', index=False)
print(f"\nEffect sizes saved to: {OUTPUT_DIR / 'effect_sizes.csv'}")

# Interpretation guide
print("\n" + "="*80)
print("Effect Size Interpretation (Cohen's d):")
print("  Small: d = 0.2")
print("  Medium: d = 0.5")
print("  Large: d = 0.8")
print("="*80)

EFFECT SIZES (Cohen's d)

Performance (z-score) - Baseline (0ms) vs Higher Latencies:
--------------------------------------------------------------------------------
0ms vs 75.0ms: d = 0.212
0ms vs 150.0ms: d = 0.674
0ms vs 225.0ms: d = 1.367

QoE Quality - Baseline (0ms) vs Higher Latencies:
--------------------------------------------------------------------------------
0ms vs 75.0ms: d = 0.896
0ms vs 150.0ms: d = 1.634
0ms vs 225.0ms: d = 2.708

Effect sizes saved to: ../analysis/results/effect_sizes.csv

Effect Size Interpretation (Cohen's d):
  Small: d = 0.2
  Medium: d = 0.5
  Large: d = 0.8


## 7. Linear Mixed-Effects Models

Properly account for within-subjects design with random effects for
participants.


In [10]:
print("=" * 80)
print("LINEAR MIXED-EFFECTS MODELS")
print("=" * 80)

# Model 1: Performance (z-score) predicted by latency
print("\nModel 1: Performance ~ Latency + (1 | Participant)")
print("-" * 80)

model1 = mixedlm(
    "score_z ~ latency_ms",
    data=df,
    groups=df["participant_id"]
)
result1 = model1.fit()
print(result1.summary())

# Save model summary
with open(OUTPUT_DIR / 'mixed_model_performance.txt', 'w') as f:
    f.write(str(result1.summary()))

# Model 2: QoE Quality predicted by latency
print("\nModel 2: QoE Quality ~ Latency + (1 | Participant)")
print("-" * 80)

qoe_valid = df.dropna(subset=['quality_rating'])
model2 = mixedlm(
    "quality_rating ~ latency_ms",
    data=qoe_valid,
    groups=qoe_valid["participant_id"]
)
result2 = model2.fit()
print(result2.summary())

with open(OUTPUT_DIR / 'mixed_model_qoe.txt', 'w') as f:
    f.write(str(result2.summary()))

# Model 3: QoE Quality predicted by latency AND performance
print("\nModel 3: QoE Quality ~ Latency + Performance + (1 | Participant)")
print("-" * 80)

perf_qoe_valid = df.dropna(subset=['quality_rating', 'score_z'])
model3 = mixedlm(
    "quality_rating ~ latency_ms + score_z",
    data=perf_qoe_valid,
    groups=perf_qoe_valid["participant_id"]
)
result3 = model3.fit()
print(result3.summary())

with open(OUTPUT_DIR / 'mixed_model_qoe_with_performance.txt', 'w') as f:
    f.write(str(result3.summary()))

LINEAR MIXED-EFFECTS MODELS

Model 1: Performance ~ Latency + (1 | Participant)
--------------------------------------------------------------------------------
         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: score_z  
No. Observations: 300     Method:             REML     
No. Groups:       17      Scale:              0.7621   
Min. group size:  8       Log-Likelihood:     -396.1750
Max. group size:  20      Converged:          Yes      
Mean group size:  17.6                                 
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.596    0.093  6.429 0.000  0.414  0.778
latency_ms   -0.005    0.001 -8.942 0.000 -0.007 -0.004
Group Var     0.025    0.030                           


Model 2: QoE Quality ~ Latency + (1 | Participant)
-------------------------------------------------------------------

## 8. Summary Statistics Table

Create a publication-ready summary table.


In [11]:
print("=" * 80)
print("SUMMARY TABLE FOR PUBLICATION")
print("=" * 80)

summary_table = df.groupby('latency_ms').agg({
    'score_z': ['mean', 'std'],
    'quality_rating': ['mean', 'std'],
    'acceptable': ['mean', 'count']
}).round(3)

# Rename columns for clarity
summary_table.columns = [
    'Performance Mean', 'Performance SD',
    'QoE Mean', 'QoE SD',
    'Acceptability %', 'N'
]

# Convert acceptability to percentage
summary_table['Acceptability %'] = (summary_table['Acceptability %'] * 100).round(1)

print(summary_table)

# Save to CSV and LaTeX format
summary_table.to_csv(OUTPUT_DIR / 'summary_table.csv')
summary_table.to_latex(OUTPUT_DIR / 'summary_table.tex')

print(f"\nSummary table saved to:")
print(f"  - {OUTPUT_DIR / 'summary_table.csv'}")
print(f"  - {OUTPUT_DIR / 'summary_table.tex'}")

SUMMARY TABLE FOR PUBLICATION
            Performance Mean  Performance SD  QoE Mean  QoE SD  \
latency_ms                                                       
0.0                    0.514           1.059     4.533   0.738   
75.0                   0.313           0.819     3.749   0.995   
150.0                 -0.176           0.987     2.803   1.309   
225.0                 -0.667           0.602     2.043   1.075   

            Acceptability %   N  
latency_ms                       
0.0                   103.9  76  
75.0                  120.0  75  
150.0                 144.6  74  
225.0                 166.2  74  

Summary table saved to:
  - ../analysis/results/summary_table.csv
  - ../analysis/results/summary_table.tex


## Summary

Statistical analysis complete! Key findings:

1. **Correlation Analysis**: Examined relationships between latency and outcomes
2. **ANOVA**: Tested significance of latency effects
3. **Post-hoc Tests**: Identified which latency pairs differ significantly
4. **Effect Sizes**: Quantified practical significance of latency impacts
5. **Mixed Models**: Properly accounted for within-subjects design

**Next steps:**

- Run `visualizations.ipynb` to create figures
- Review all output files in `analysis/results/`
- Interpret findings in the context of your research questions
